# Setup

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# import libraries

In [ ]:
import pandas as pd
import numpy as  np
import plotly.express as ex
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score,classification_report
from warnings import filterwarnings
from plotly.subplots import make_subplots
import plotly.graph_objects as go

filterwarnings('ignore')

# getting data

In [ ]:
df = pd.read_csv('/kaggle/input/credit-card-customers/BankChurners.csv').drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1','CLIENTNUM','Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'],axis=1)
print('shape: {} '.format(df.shape))
df.head()

# data exam

In [ ]:
df.info()

In [ ]:
df.isna().sum().sum()

In [ ]:
df.Attrition_Flag.unique() # dependend value count

In [ ]:
df.select_dtypes(['object']).nunique()

In [ ]:
# categoric features
cat = df.select_dtypes(['object'])
cat_columns = cat.columns
cat_columns

In [ ]:
# numeric features
numeric_values = df.select_dtypes(['int','float'])

In [ ]:
print(cat.shape)
print(cat.info())
cat.head()

In [ ]:
cat.nunique()

In [ ]:
print(numeric_values.shape)
print(numeric_values.info())
numeric_values.head()

# data visulation 

In [ ]:
sns.countplot(df.Attrition_Flag)

In [ ]:
cstage = numeric_values.groupby(['Customer_Age']).aggregate('mean') # group the age 

In [ ]:
cstage.head()

In [ ]:
fig = make_subplots(
    rows=5, cols=1,
    subplot_titles=("Relationship between Age and Credit Card limit", "Relationship between Age and Dependent Count", "Relationship between Age and Avg_Open_To_Buy", 
                    "Relationship between Age and Total_Revolving_Bal","Relationship between Age and Contacts_Count_12_mon"
                    ))

fig.add_trace(go.Scatter(x=cstage.index, y=cstage.Credit_Limit,mode='lines+markers' ),
              row=1, col=1)

fig.add_trace(go.Bar(x=cstage.index, y=cstage.Dependent_count),
              row=2, col=1)

fig.add_trace(go.Scatter(x=cstage.index, y = cstage.Avg_Open_To_Buy,marker={'color': 'blue'},mode = 'lines+markers'),
              row=3, col=1)

fig.add_trace(go.Bar(x=cstage.index, y=cstage.Total_Revolving_Bal, marker  = {'color':'gray'} ),
              row=4,col=1)
fig.add_trace(go.Scatter(x = cstage.index ,y = cstage.Contacts_Count_12_mon, marker = {'color':'magenta'},mode = 'lines+markers'),
             row=5,col=1)

fig.update_layout(height=1200, width=900,
                  title_text="Relationship between Age and other features")

fig.show()

In [ ]:
cat.head()

In [ ]:
cat.Attrition_Flag.unique()

In [ ]:
gender = df.groupby(['Gender']).aggregate('mean')# group the gender

In [ ]:
gender

In [ ]:
fig = ex.pie(df, values='Credit_Limit', names='Gender', title='Relationship between Gender and Credit Limit ')
fig.show()

In [ ]:
fig = ex.pie(df, values='Credit_Limit', names='Education_Level', title='Relationship between Education Level and Credit Limit ')
fig.show()

In [ ]:
fig = ex.histogram(y=df.Credit_Limit, x=df.Marital_Status,title = "Relationship between Marital Status and Credit Card Limid " ) 
fig.show()

# Preprocessing 

In [ ]:
le = LabelEncoder()

In [ ]:
for i in cat.columns:
    cat[i] = le.fit_transform(cat[[i]])
cat

In [ ]:
df_reduced = pd.concat([numeric_values,cat],axis=1)

In [ ]:
df_reduced.head()

# train-test split 

In [ ]:
x = df_reduced.drop(['Attrition_Flag'],axis=1)
y = df_reduced[["Attrition_Flag"]]

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.25,random_state = 14,shuffle = True)

# Model

In [ ]:
grd = GradientBoostingClassifier(n_estimators = 999).fit(x_train,y_train)

In [ ]:
def metrics(y_true,y_pred):
    print(classification_report(y_true,y_pred))
    print("Accuracy: {}".format(accuracy_score(y_true,y_pred)))

In [ ]:
prediction = grd.predict(x_test)
metrics(y_test,prediction)